# 初始化

In [1]:
from alpha.notebook import *

await init_notebook()

# 涨跌幅相似法

取最后n bars的涨跌幅进行学习和预测

In [18]:
async def build_dataset():
    start = tf.day_shift(arrow.get("2021-01-04"), 0)
    end = tf.day_shift(arrow.get("2021-08-31"), 0)
    
    X = []
    y = []
    
    for code in Securities().choose(["stock"]):
        for frame in tf.get_frames(start, end, FrameType.DAY):
            sec = Security(code)
            end_ = tf.int2date(frame)
            start_ = tf.day_shift(end_, -16)
            
            bars = await sec.load_bars(start_, end_, FrameType.DAY)
            close = bars["close"]
            if len(close) < 17 or np.count_nonzero(np.isfinite(close)) < 17 * 0.9:
                continue
                
            close = fillna(close.copy())
            
            pcr = close[1:]/close[:-1] - 1
            
            X.append(np.round(pcr[:-1], 3))
            y.append(pcr[-1])
            
        if len(y) == 10000:
            break
    return X, y

In [20]:
X, y = await build_dataset()

TimeoutError: 